In [1]:
# encoding = 'utf-8'

import pandas as pd

data = pd.read_csv('Online_Retail.csv')

In [2]:
data.head()

,Unnamed: 0,TransactionNo,MerchandiseCode,Quan,TransactionDate,ItemPrice,UserID
0,0,29138841,85123A,8,2022-09-01,2.76,208163.0
1,1,29138841,71053,8,2022-09-01,2.12,208163.0
2,2,29138841,84406B,8,2022-09-01,5.12,208163.0
3,3,29138841,84029G,8,2022-09-01,4.33,208163.0
4,4,29138841,84029E,8,2022-09-01,3.39,208163.0


# 1.从交易明细数据中找到所有用户的前30个最频繁购买项

In [3]:
data[['MerchandiseCode','UserID']].drop_duplicates().groupby('MerchandiseCode').count().rename(columns={'UserID':'count'}).sort_values(by='count',ascending=False).head(30)


,count
MerchandiseCode,
22423,887
85123A,858
47566,708
84879,679
22720,640
21212,636
85099B,636
22086,615
22457,590


# 2.依据apriori 算法对商品之间的关联程度/规则 进行一定程度的挖掘

In [10]:
all_df.head()

,UserID,TransactionNo,MerchandiseCode
0,208163.0,29138841,"85123A,71053,84406B,84029G,84029E,22752,21730"
1,208163.0,29326612,"22633,22632"
2,212862.0,291388311,"84879,22745,22748,22749,22310,84969,22623,2262..."
3,212862.0,22345123,"22960,22913,22912,22914"
4,212862.0,536369,21756


In [14]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

#设置数据集
dataset = list(all_df['MerchandiseCode'])
dataset = [item.split(',') for item in dataset]
te = TransactionEncoder()
#进行 one-hot 编码
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出频繁项集
freq = apriori(df, min_support=0.01, use_colnames=True)

In [15]:
from mlxtend.frequent_patterns import association_rules

association_rules(freq,metric='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(85099B),(20712),0.063436,0.019344,0.010232,0.161290,8.338162,0.009005,1.169244
1,(20712),(85099B),0.019344,0.063436,0.010232,0.528942,8.338162,0.009005,1.988214
2,(20719),(20724),0.020811,0.029151,0.011197,0.538033,18.457040,0.010590,2.101558
3,(20724),(20719),0.029151,0.020811,0.011197,0.384106,18.457040,0.010590,1.589866
4,(20723),(20724),0.019459,0.029151,0.011583,0.595238,20.419426,0.011016,2.398569
...,...,...,...,...,...,...,...,...,...
353,"(22726, 22728)",(22727),0.013282,0.035019,0.010347,0.779070,22.246865,0.009882,4.367807
354,"(22727, 22728)",(22726),0.015560,0.031236,0.010347,0.665012,21.290261,0.009861,2.891941
355,(22726),"(22727, 22728)",0.031236,0.015560,0.010347,0.331273,21.290261,0.009861,1.472111
356,(22727),"(22726, 22728)",0.035019,0.013282,0.010347,0.295480,22.246865,0.009882,1.400553


# 3.分析哪些商品可以一起购买，并计算得出商品组合之间的支持度

In [22]:
freq['item_len'] = freq['itemsets'].apply(lambda x:len(x))

In [23]:
freq[freq['item_len'] > 1].sort_values(by='support',ascending=False)

,support,itemsets,item_len
512,0.021506,"(22699, 22697)",2
496,0.021390,"(85099B, 22386)",2
515,0.020695,"(22726, 22727)",2
427,0.020463,"(20725, 22384)",2
426,0.020309,"(20725, 22383)",2
...,...,...,...
429,0.010077,"(20725, 23203)",2
553,0.010039,"(22382, 20725, 20726)",3
469,0.010039,"(21929, 22411)",2
441,0.010039,"(23209, 20726)",2
